# **Regresja wieloraka**

## Wczytywanie danych symulacji

In [39]:
import pandas as pd

df = pd.read_csv("symulacja_banku.csv")  # lub symulacja_banku_godziny.csv, zależnie od pliku
df['Dzien'] = df['Dzien'].astype(int)
dzien_encoded = pd.get_dummies(df['Dzien'], prefix='Dzien')
df_encoded = pd.concat([df, dzien_encoded], axis=1)

display(df_encoded)

,ID,Czas_przyjscia,Dlugosc kolejki w momencie przyjscia,Czas_rozpoczecia_obslugi,Czas_zakonczenia_obslugi,Czas_oczekiwania,Dzien,Stanowisko,Dzien_1,Dzien_2,Dzien_3,Dzien_4,Dzien_5
0,1001,0,0,0,8,0,1,1,True,False,False,False,False
1,1002,2,0,2,20,0,1,2,True,False,False,False,False
2,1003,3,0,3,5,0,1,3,True,False,False,False,False
3,1004,6,0,6,12,0,1,3,True,False,False,False,False
4,1005,17,0,17,23,0,1,1,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
957,5186,465,1,465,467,0,5,2,False,False,False,False,True
958,5187,465,0,465,471,0,5,3,False,False,False,False,True
959,5188,468,0,468,475,0,5,1,False,False,False,False,True
960,5189,471,0,471,483,0,5,2,False,False,False,False,True


In [40]:
def prepare_data(df):
    X = []
    y = []

    for _, row in df.iterrows():
        features = [
            1.0,  # bias
            float(row['Czas_przyjscia']),
            float(row['Dlugosc kolejki w momencie przyjscia']),
        ]
        # Tylko 4 kolumny zamiast 5
        for i in range(1, 5):  # Dzien_1 do Dzien_4
            features.append(int(row.get(f'Dzien_{i}', 0)))

        X.append(features)
        y.append([float(row['Czas_oczekiwania'])])

    return X, y


In [41]:
import numpy as np

def transpose(m):
    return list(map(list, zip(*m)))

def matmul(A, B):
    return [[sum(a * b for a, b in zip(row, col)) for col in zip(*B)] for row in A]

def inverse(matrix):
    return np.linalg.inv(np.array(matrix)).tolist()

def regression_beta(X, y):
    Xt = transpose(X)
    XtX = matmul(Xt, X)
    XtX_inv = inverse(XtX)
    XtY = matmul(Xt, y)
    return matmul(XtX_inv, XtY)

def predict(X, beta):
    return [sum(xi * bi[0] for xi, bi in zip(row, beta)) for row in X]

def r_squared(y_true, y_pred):
    mean_y = sum(y_true) / len(y_true)
    ss_tot = sum((yi - mean_y) ** 2 for yi in y_true)
    ss_res = sum((yi - ypi) ** 2 for yi, ypi in zip(y_true, y_pred))
    return 1 - ss_res / ss_tot




In [42]:
X, y = prepare_data(df_encoded)

beta = regression_beta(X, y)
y_hat = predict(X, beta)
r2 = r_squared([val[0] for val in y], y_hat)

print("Współczynniki beta:")
for i, b in enumerate(beta):
    print(f"b{i} = {b[0]}")

print(f"\nR² dopasowania modelu: {r2}")


Współczynniki beta:
b0 = -2.4483872175631163
b1 = 0.012526067507090264
b2 = 2.1989904846426307
b3 = -0.14591495651726727
b4 = 0.051899307493039615
b5 = 4.087509657621749
b6 = -0.23763525537061092

R² dopasowania modelu: 0.8749074119924807


## **Współczynniki beta**

**b0**​	- wyraz wolny	-2.45 – wartość bazowa (czas oczekiwania), gdy wszystkie inne zmienne są zerowe. <br>
 Tutaj ma mały sens praktyczny, bo 0 minut przyjścia i 0 długości kolejki są rzadkie, a dodatkowo to tylko punkt odniesienia.

**b1**​	- Czas przyjścia	0.0125 – każda dodatkowa minuta przyjścia zwiększa czas oczekiwania o ok. 0.013 minuty, czyli 0.75 sekundy. Bardzo mały wpływ.

**b2**​	- Długość kolejki	2.20 – każda dodatkowa osoba w kolejce oznacza średnio 2.2 minuty więcej oczekiwania. To istotny współczynnik.

**b3​**	- Dzień 1	-0.15 – w porównaniu do dnia 5, dzień 1 daje nieco mniejsze oczekiwanie.

**b4**​	- Dzień 2	+0.05 – znikomy wzrost względem dnia 5.

**b5**​	- Dzień 3	+4.09 – duży wzrost oczekiwania względem dnia 5 – średnio o ponad 4 minuty.

**b6**​	- Dzień 4	-0.24 – dzień 4 jest trochę bardziej "luźny" niż dzień 5.

**Co z tego wynika?**

* Największy wpływ ma długość kolejki.
* Dzień 3 znacznie zwiększa czas oczekiwania – być może to dzień z największym ruchem.
* Pozostałe dni nie różnią się drastycznie od dnia 5 (referencyjnego).
* Czas przyjścia ma znikomy wpływ – system obsługuje klientów według kolejki, nie czasu przyjścia.

## **R²**

**R² = 0.875** → model dobrze dopasowany do danych.

Mówiąc inaczej: 87,5% różnic w czasie oczekiwania da się wyjaśnić przez zmienne wejściowe (np. długość kolejki, dzień, stanowisko itd.).

Pozostałe 12,5% to „szum”, czyli coś, czego model nie przewidział – np. losowe fluktuacje, czynniki nieuwzględnione w modelu.